# Conventions

This tutorial lists some of the most important conventions and assumptions used in PorePy.

Specifically, we cover conventions for:
* Geometry
* Boundaries
* Equations
* Apertures
* Coupling between dimensions

## Geometry
Normal vectors are defined once for each face and can be accessed through `sd.face_normals` for a subdomain, `sd`. These vectors are weighted with the face area, meaning that the area of a face can be found by taking the norm of its normal vector.

For a subdomain grid sd, the field `sd.cell_faces` is the discrete divergence: 
* It has dimensions (number of faces) $\times$ (number of cells)
* It is nonzero only for the neighboring faces of any given cell
* The value is positive if the normal vector of the face points outwards from the cell and negative otherwise.

See [this](./grid_topology.ipynb) tutorial for a demonstrations of the aforementioned.


This can be used to obtain the outwards pointing normal vector of a cell. We simply multiply with the value ($\pm 1$) of sd.cell_faces. For instance, for cell `c` and face `f` we have

`outward_normal = sd.cell_faces[f, c] * sd.face_normals[:, f]`
### Local coordinates and internal boundaries 
In [mixed-dimensional grids](./mixed_dimensional_grids.ipynb) (TODO: Move there? Or create fracture deformation tutorial?), fractures define internal boundaries for the matrix.
Each fracture is assigned a local basis with a normal vector and one or two tangential vectors for ambient dimension of 2 and 3, respectively. 
The fracture's normal vector coincides with the outwards normal vector of the matrix on the "j side"/"left side" of the fracture (the other side is referred to as the "k side" or the "right side").
The fracture is assigned a `TangentialNormalProjection` with methods to rotate between global and local coordinates.

In some settings, it is important to distinguish between the two sides of a fracture.
This information is contained in the `sign_of_mortar_sides` method of the `MortarGrid` representing the fracture-matrix interface.
The method returns a matrix with $-1$ and $+1$ entries corresponding to the j and k side, respectively.


## External boundaries
### Flow
Outflow values are considered positive for Neumann type boundary conditions, i.e. outward pointing normal vectors are assumed.

On inner boundaries, a positive mortar flux $\lambda$ implies flow from the higher-dimensional to the lower-dimensional domain.
### Mechanics
Mechanical boundary condtions are always given in global coordinates. 
For Dirichlet boundary condition values, we prescribe the displacement values directly without regarding the normal vector. 
For a 2d  domain, this means that a prescribed boundary value of $\mathbf{u}_b = [1, -2]$ signifies a unitary displacement to the right, and downwards with magnitude 2. 
Similarly, $\sigma\cdot \mathbf{n} = [-1, 0]$ implies a unitary force to the left.

Displacements defined on inner boundaries are also in global coordinates.


## Equations
### Momentum balance
The momentum balance equation within PorePy follows the convention of positive tensile stress. 
In practice this means that the equation is described by the following formulation:

$\frac{\partial^2 u}{\partial t^2} = \nabla \cdot \sigma (\epsilon (u)) + q$,

where $u$ is displacement, $\sigma(\epsilon(u))$ is the stress tensor and $q$ is the body force.
Usually the stress tensor is related to displacement through Hooke's law and the strain-displacement relation for small deformations:

$\sigma(\epsilon(u)) = C : \epsilon(u) = C : \frac{1}{2} (\nabla u + (\nabla u)^T)$,

where $C$ denotes the stiffness tensor.

### Spatial dimensions
PorePy allows simulations in 3, 2 and, for some cases, 1 dimensions. We denote the highest dimension by `nd`. If `nd`$<3$, the units of the equation change accordingly - e.g. kg/s/m for a `nd`=2 mass balance. This implies that the units of parameters entering the equation remain unchanged. For instance, the mass density is still kg/m$^3$, not kg/m$^2$.

### Nondimensionalization
Currently, most equations and variables have their natural physical units.
The exceptions are the contact mechanics equations and the contact traction, which are nondimensionalized using a characteristic traction.

## Apertures
In our mixed-dimensional models we denote the thickness of the fractures by the term `aperture` [m], denoted by $a$. 

Volumes [m$^3$] of fracture or intersection cells are computed through their specific volumes. 
In any given dimension d, this is the volume of the cell per d-dimensional measure. 
In 2d, it is the volume per area, and has dimensions [m$^3/$m$^2$] = [m]. 
The measure is a line measure for 1d intersection lines, and the specific volume thus has the dimensions [m$^3/$m] = [m$^2$]. 
In general, the dimensions are [m$^{3-d}$], where d is the dimension of the subdomain in question.
The full volume is always the product of the d-dimensional cell volume and the specific volume.

`volumes = g.cell_volumes * specific_volumes` $\quad$ [m$^3$]

This implies that some parameters should be weighted by the specific volumes. 
This holds for tangential permeability, volumetric source terms and several other parameters. 
To be clear, the permeabilities specified within and between subdomains are absolute, that is, no aperture scaling of permeabilities takes place inside the code.